In [1]:
import requests
import json
import csv
import pandas as pd
import urllib.parse
import numpy as np

pd.set_option('display.max_columns', None)       # Show all columns
pd.set_option('display.width', None)

In [9]:
in_file = '2020_fires_in_tempe_loc1.csv'
out_file = '2020_fires_in_tempe.json'

In [5]:
with open('secrets/gmaps_key', 'r') as f:
    api_key = f.read()

In [6]:
with open(in_file,'r') as f:
    df = pd.read_csv(f)

# df = df.iloc[:,1:]
# df['_addr'] = df['_addr'].map(lambda val: urllib.parse.quote(val))
df

responses = pd.read_pickle('api_response.pkl')

In [8]:
responses = []
url = 'https://maps.googleapis.com/maps/api/geocode/json'

for i,row in df.iterrows():
    params = {
        'key': api_key,
        'outputFormat': 'json',
        'address': row['_addr']
    }
    print(row['_addr'])
    response = requests.get(url, params=params)
    if response.status_code == 200:
        responses.append(response.json())
        print(response.json())
    else:
        print(f"Error {response.status_code}")
    # break

pd.to_pickle(responses, 'api_response.pkl')

with open('api_response.json','w') as f:
    f.write(json.dumps(responses))

1133 W BASELINE RD Tempe AZ 85283
{'results': [{'address_components': [{'long_name': '1133', 'short_name': '1133', 'types': ['street_number']}, {'long_name': 'West Baseline Road', 'short_name': 'W Baseline Rd', 'types': ['route']}, {'long_name': 'South Tempe', 'short_name': 'South Tempe', 'types': ['neighborhood', 'political']}, {'long_name': 'Tempe', 'short_name': 'Tempe', 'types': ['locality', 'political']}, {'long_name': 'Maricopa County', 'short_name': 'Maricopa County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Arizona', 'short_name': 'AZ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '85283', 'short_name': '85283', 'types': ['postal_code']}, {'long_name': '5351', 'short_name': '5351', 'types': ['postal_code_suffix']}], 'formatted_address': '1133 W Baseline Rd, Tempe, AZ 85283, USA', 'geometry': {'location': {'lat': 33.3779413, 'lng': -111.95

In [10]:
lat = []
lng = []
for r in responses:
    lat.append(r['results'][0]['geometry']['location']['lat'])
    lng.append(r['results'][0]['geometry']['location']['lng'])

In [11]:
df['lat'] = lat
df['lng'] = lng
df

,INCIDENT_KEY,_addr,_desc,INC_DATE,lat,lng
0,AZ_08293_01102020_20839_0,1133 W BASELINE RD Tempe AZ 85283,Number of Residential Units: 0.0\nArea of Orig...,2020-01-10,33.377941,-111.955429
1,AZ_08293_01302020_202468_0,5250 S HARDY DR Tempe AZ 85283,Number of Residential Units: 4.0\nNumber of Bu...,2020-01-30,33.376296,-111.954183
2,AZ_08293_01312020_202554_0,607 W 19TH ST Tempe AZ 85281,Number of Residential Units: 10.0\nAcres Burne...,2020-01-31,33.408064,-111.948334
3,AZ_08293_01312020_202523_0,323 E VETERANS Tempe AZ 85281,Number of Residential Units: 400.0\nArea of Or...,2020-01-31,33.424998,-111.934388
4,AZ_08293_01312020_202507_0,1210 E GILBERT DR Tempe AZ 85281,Number of Residential Units: 1.0\nArea of Orig...,2020-01-31,33.437200,-111.919995
...,...,...,...,...,...,...
224,AZ_08293_12242020_2028617_0,843 E DIVOT DR Tempe AZ 85283,Number of Residential Units: 1.0\nArea of Orig...,2020-12-24,33.355383,-111.929072
225,AZ_08293_12252020_2028720_0,2134 E BROADWAY RD Tempe AZ 85281,Number of Residential Units: 16.0\nNumber of B...,2020-12-25,33.407895,-111.895443
226,AZ_08293_12312020_2029122_0,1445 W ELLIOT RD Tempe AZ 85284,Not Residential Flag: Y\nOn Site Materials #1:...,2020-12-31,33.347473,-111.962763
227,AZ_08293_12312020_2029158_0,1835 E GUADALUPE RD Tempe AZ 85283,"On Site Materials #1: Food, other\nArea of Ori...",2020-12-31,33.362147,-111.909142


In [12]:
import geojson

features = []

for i, row in df.iterrows():
    p = geojson.Point((row['lat'], row['lng']))
    f = geojson.Feature(id=i, geometry=p, properties={
        "description": row['_desc'],
        "date": row['INC_DATE'],
        "address": row['_addr']
    })
    features.append(f)

fc = geojson.FeatureCollection(features=features)

with open(out_file,'w') as f:
    f.write(geojson.dumps(fc))

In [ ]:
# responses = []
# url = 'https://maps.googleapis.com/maps/api/elevation/json'

# # for i, row in df.iterrows():
# #     params = {
# #         'key': api_key,
# #         'locations': f"{str(row['lat'])}, {str(row['lng'])}"
# #     }
# #     response = requests.get(url, params=params)
# #     if response.status_code == 200:
# #         responses.append(response.json())
# #         print(response.json())
# #     else:
# #         print(f"Error {response.status_code}")

{'results': [{'elevation': 357.6151733398438, 'location': {'lat': 33.3245334, 'lng': -111.9308314}, 'resolution': 9.543951988220215}], 'status': 'OK'}
{'results': [{'elevation': 361.8256530761719, 'location': {'lat': 33.3644015, 'lng': -111.9089034}, 'resolution': 9.543951988220215}], 'status': 'OK'}
{'results': [{'elevation': 364.9025268554688, 'location': {'lat': 33.4461989, 'lng': -111.9269811}, 'resolution': 9.543951988220215}], 'status': 'OK'}
{'results': [{'elevation': 368.1124877929688, 'location': {'lat': 33.3594109, 'lng': -111.9486624}, 'resolution': 9.543951988220215}], 'status': 'OK'}
{'results': [{'elevation': 360.7393493652344, 'location': {'lat': 33.3902572, 'lng': -111.9194455}, 'resolution': 9.543951988220215}], 'status': 'OK'}
{'results': [{'elevation': 356.8144836425781, 'location': {'lat': 33.4259451, 'lng': -111.9367275}, 'resolution': 9.543951988220215}], 'status': 'OK'}
{'results': [{'elevation': 359.0735168457031, 'location': {'lat': 33.3837369, 'lng': -111.9414

In [ ]:
# elevations = []
# for row in responses:
#     elevations.append(row['results'][0]['elevation'])

In [ ]:
# df['elevation'] = elevations

In [ ]:
# df.to_json('nfirs_cleaned_forunity.json',orient='records')